### Imports

In [55]:
import pandas as pd

In [56]:
df = pd.read_csv("./data/frmgham2_dirty_modified.csv")
df.head()

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,...,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,2448,1,195.0,39,106.0,70.0,No,0.0,26.97,No,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
1,2448,1,209.0,52,121.0,66.0,No,0.0,NaN,No,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
2,6238,2,250.0,46,121.0,81.0,No,0.0,28.73,No,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
3,6238,2,260.0,52,105.0,69.5,No,0.0,29.43,No,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
4,6238,2,237.0,58,108.0,66.0,No,0.0,28.50,No,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766


### Modification

Make a clean copy.

In [57]:
df_no_missing = df.copy()
df_no_missingn = df_no_missing.dropna()
df_no_missing = df_no_missing[(df_no_missing["educ"] != 0) & (df_no_missing["AGE"] != -1)]
df_no_missing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11224 entries, 0 to 11631
Data columns (total 39 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   RANDID    11224 non-null  int64  
 1   SEX       11224 non-null  object 
 2   TOTCHOL   10834 non-null  float64
 3   AGE       11224 non-null  int64  
 4   SYSBP     11224 non-null  float64
 5   DIABP     11224 non-null  float64
 6   CURSMOKE  11224 non-null  object 
 7   CIGPDAY   11151 non-null  float64
 8   BMI       11174 non-null  float64
 9   DIABETES  11224 non-null  object 
 10  BPMEDS    10644 non-null  float64
 11  HEARTRTE  11218 non-null  float64
 12  GLUCOSE   9822 non-null   float64
 13  educ      11224 non-null  float64
 14  PREVCHD   11224 non-null  int64  
 15  PREVAP    11224 non-null  int64  
 16  PREVMI    11224 non-null  int64  
 17  PREVSTRK  11224 non-null  int64  
 18  PREVHYP   11224 non-null  int64  
 19  TIME      11224 non-null  int64  
 20  PERIOD    11224 non-null  in

#### Preparation For Association Rules

In [58]:
df_ar = df_no_missing.copy()
df_ar = df_ar[["RANDID", "PERIOD", "DIABETES", "STROKE", "HYPERTEN", "ANYCHD"]]
df_ar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11224 entries, 0 to 11631
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   RANDID    11224 non-null  int64 
 1   PERIOD    11224 non-null  int64 
 2   DIABETES  11224 non-null  object
 3   STROKE    11224 non-null  int64 
 4   HYPERTEN  11224 non-null  int64 
 5   ANYCHD    11224 non-null  object
dtypes: int64(4), object(2)
memory usage: 613.8+ KB


In [59]:
df_ar = df_ar[df_ar["PERIOD"] == 3]
df_ar = df_ar.drop("PERIOD", axis = 1)
df_ar.head()

,RANDID,DIABETES,STROKE,HYPERTEN,ANYCHD
1,2448,No,0,0,Yes
4,6238,No,0,0,No
11,11252,No,0,1,No
14,11263,Yes,0,1,Yes
19,12806,No,0,1,No


In [60]:
def create_transactions(random_seed):
  df_50 = df_ar.sample(50, random_state=random_seed)

  df_ar_trans = pd.DataFrame()

  for index, data in df_50.iterrows():
    if data["DIABETES"] == "Yes":
      df_ar_trans = pd.concat([df_ar_trans, pd.DataFrame({ "RANDID": [data["RANDID"]], "Condition": ["DIABETES"] })])
    if data["STROKE"] == 1:
      df_ar_trans = pd.concat([df_ar_trans, pd.DataFrame({ "RANDID": [data["RANDID"]], "Condition": ["STROKE"] })])
    if data["HYPERTEN"] == 1:
      df_ar_trans = pd.concat([df_ar_trans, pd.DataFrame({ "RANDID": [data["RANDID"]], "Condition": ["HYPERTEN"] })])
    if data["ANYCHD"] == "Yes":
      df_ar_trans = pd.concat([df_ar_trans, pd.DataFrame({ "RANDID": [data["RANDID"]], "Condition": ["ANYCHD"] })])

  df_ar_trans = df_ar_trans.reset_index(drop=True)

  return df_50, df_ar_trans

In [61]:
df_ar_sample_a, df_ar_trans_sample_a = create_transactions(43)

In [62]:
df_ar_sample_b, df_ar_trans_sample_b = create_transactions(52)

In [63]:
df_ar_sample_c, df_ar_trans_sample_c = create_transactions(64)

In [64]:
df_ar_sample_d, df_ar_trans_sample_d = create_transactions(72)

### Exports

In [65]:
df_ar_sample_a.to_csv("./data/frmgham2_ar_sample_a.csv", index=False)

In [66]:
df_ar_sample_b.to_csv("./data/frmgham2_ar_sample_b.csv", index=False)

In [67]:
df_ar_sample_c.to_csv("./data/frmgham2_ar_sample_c.csv", index=False)

In [68]:
df_ar_sample_d.to_csv("./data/frmgham2_ar_sample_d.csv", index=False)

In [69]:
df_ar_trans_sample_a.to_csv("./data/frmgham2_ar_trans_sample_a.csv", index=False)

In [70]:
df_ar_trans_sample_b.to_csv("./data/frmgham2_ar_trans_sample_b.csv", index=False)

In [71]:
df_ar_trans_sample_c.to_csv("./data/frmgham2_ar_trans_sample_c.csv", index=False)

In [72]:
df_ar_trans_sample_d.to_csv("./data/frmgham2_ar_trans_sample_d.csv", index=False)